## Cell Segmentation

In [ ]:
import os

import bioimageio.core
import h5py
import napari

from skimage.segmentation import watershed
from xarray import DataArray

In [ ]:
data_folder = "../data"
output_folder = os.path.join(data_folder, "predictions")

### 1. Implement Cell Segmentation

In [ ]:
model_path = os.path.join(data_folder, "trained_models/boundary-segmentation/boundary_segmentation_model.zip")
model = bioimageio.core.load_resource_description(model_path)

In [ ]:
image_path = os.path.join(data_folder, "test/gt_image_048.h5")
prediction_path = os.path.join(data_folder, "predictions/gt_image_048.h5")

with h5py.File(image_path, "r") as f:
    image = f["raw/serum_IgG/s0"][:]
    
with h5py.File(prediction_path, "r") as f:
    nuclei = f["/segmentations/nuclei/watershed_based"][:]

In [ ]:
with bioimageio.core.create_prediction_pipeline(model) as pp:
    input_ = DataArray(image[None, None], dims=tuple("bcyx"))
    pred = bioimageio.core.predict_with_padding(pp, input_, padding={"x": 16, "y": 16})[0].values.squeeze()

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_image(pred)
viewer.add_labels(nuclei)

In [ ]:
foreground, boundaries = pred
foreground = foreground > 0.5
cells = watershed(boundaries, markers=nuclei, mask=foreground)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(cells)

### 2. Apply to Test Images

In [ ]:
from glob import glob
from tqdm import tqdm

test_images = glob(os.path.join(data_folder, "test/*.h5"))

In [ ]:
def segment_cells(pp, image, nuclei):
    input_ = DataArray(image[None, None], dims=tuple("bcyx"))
    pred = bioimageio.core.predict_with_padding(pp, input_, padding={"x": 16, "y": 16})[0].values.squeeze()
    foreground, boundaries = pred
    foreground = foreground > 0.5
    cells = watershed(boundaries, markers=nuclei, mask=foreground)
    return cells

In [ ]:
with bioimageio.core.create_prediction_pipeline(model) as pp:
    for path in tqdm(test_images):
        out_path = os.path.join(output_folder, os.path.basename(path))
        with h5py.File(path, "r") as f:
            image = f["raw/serum_IgG/s0"][:]
        with h5py.File(out_path, "r") as f:
            nuclei = f["segmentations/nuclei/watershed_based"][:]
        cells = segment_cells(pp, image, nuclei)
        with h5py.File(out_path, "a") as f:
            f.create_dataset("segmentations/cells/watershed_based", data=cells, compression="gzip")

### Exercises

Lorem ipsum

**What's next**